In [ ]:
# SORU 1
SELECT
	TITLE + ' (' + CONVERT(VARCHAR, YEAR(RELEASE_DATE)) + ')' AS TITLE,
	CONVERT(VARCHAR, RUNTIME / 60) + 'h ' + CONVERT(VARCHAR, RUNTIME % 60) + 'm' AS RUNTIME,
	VOTE_AVERAGE * 10 AS RATING, 
	TAGLINE,
	OVERVIEW,
	POSTER_PATH,
	(
		SELECT GENRE + ',' 
		FROM MOVIE_GENRE AS MG
		JOIN GENRE AS G ON G.GENREID = MG.GENREID
		WHERE MG.FILMID = M.FILMID
		FOR XML PATH ('')
	) AS GENRES,
	(
		SELECT 
			(P.NAME_ + ' (' + J.JOB + ')') + ','
		FROM CREW_CREDIT AS CC
		JOIN DEPARTMENT AS D ON D.ID = CC.DEPARTMENTID
		JOIN JOB AS J ON J.ID = CC.JOBID
		JOIN PERSON AS P ON P.PERSONID = CC.PERSONID
		WHERE 
			FILMID = M.FILMID 
			AND J.JOB IN ('Director', 'Screenplay')
		FOR XML PATH ('')
	) AS CREW
FROM MOVIE AS M
WHERE FILMID = 98

In [ ]:
# SORU 2
SELECT TOP 1 
	TITLE,
	STATUS_,
	LA.NAME,
	MO.BUDGET,
	MO.REVENUE,
	(
		SELECT TOP 10
			KEYWORD + ', '
		FROM MOVIE_KEYWORD AS MK
		WHERE MK.FILMID = MO.FILMID
		FOR XML PATH ('')
	) AS KEYWORD,
	(
		SELECT TOP 6
			CHARACTER_ + ', '
		FROM CAST_CREDIT AS CC
		JOIN PERSON AS PE ON PE.PERSONID = CC.PERSONID
		WHERE CC.FILMID = MO.FILMID
		FOR XML PATH ('')
	) AS CHARACTERS,
	(
		SELECT TOP 6
			PE.NAME_ + ', '
		FROM CAST_CREDIT AS CC
		JOIN PERSON AS PE ON PE.PERSONID = CC.PERSONID
		WHERE CC.FILMID = MO.FILMID
		FOR XML PATH ('')
	) AS CHARACTERS
FROM MOVIE AS MO
JOIN LANGUAGE_ AS LA ON LA.CODE = MO.ORIGINAL_LANGUAGE
WHERE 
	MO.FILMID = 98

In [ ]:
# SORU 3
SELECT
	PE.PERSONID,
	PE.NAME_,
	BIOGRAPHY,
	DEPARTMENT, 
	CASE
		WHEN PE.GENDER = 1 THEN 'FEMALE'
		WHEN PE.GENDER = 2 THEN 'MALE'
	END AS GENDER,
	BIRTHDAY,
	PLACE_OF_BIRTH,
	ALSO_KNOWN_AS,
	CC.CHARACTER_,
	MO.TITLE
FROM PERSON AS PE
JOIN CAST_CREDIT AS CC ON CC.PERSONID = PE.PERSONID
LEFT JOIN MOVIE AS MO ON MO.FILMID = CC.FILMID
WHERE PE.NAME_ LIKE 'Denzel Washin%'

In [ ]:
# SORU 4
SELECT 
	CO.NAME_ AS [FİLM SERİSİ],
	COUNT(*) AS [TOPLAM FİLM],
	SUM(MO.BUDGET) AS [TOPLAM BÜTÇE],
	ROUND(AVG(MO.BUDGET), 0) AS [ORTALAMA BÜTÇE],
	SUM(MO.REVENUE) AS [TOPLAM HASILAT],
	ROUND(AVG(MO.REVENUE), 0) AS [ORTALAMA HASILAT]
FROM MOVIE AS MO
JOIN COLLECTION_ AS CO ON CO.COLLECTIONID = MO.COLLECTIONID
GROUP BY CO.NAME_
HAVING SUM(MO.BUDGET) > 0
ORDER BY 3 DESC

In [ ]:
# SORU 5
SELECT 
	CO.COLLECTIONID,
	CO.NAME_,
	(SELECT SUM(BUDGET) 
	FROM MOVIE 
	WHERE COLLECTIONID = CO.COLLECTIONID) AS SERIBUTCESI,
	(SELECT COUNT(*) 
	FROM MOVIE 
	WHERE COLLECTIONID = CO.COLLECTIONID) AS FILMSAYISI,
	(SELECT TOP 1 CONVERT(VARCHAR, YEAR(RELEASE_DATE)) + ' - ' + TITLE + ' - ' + FORMAT(BUDGET, '0') 
	FROM MOVIE 
	WHERE COLLECTIONID = CO.COLLECTIONID 
	ORDER BY BUDGET DESC) AS [ENYUKSEKBUTCE],
	(SELECT TOP 1 CONVERT(VARCHAR, YEAR(RELEASE_DATE)) + ' - ' + TITLE + ' - ' + FORMAT(BUDGET, '0') 
	FROM MOVIE 
	WHERE COLLECTIONID = CO.COLLECTIONID 
	ORDER BY BUDGET) AS [ENDUSUKBUTCE],
	(SELECT TOP 1 CONVERT(VARCHAR, YEAR(RELEASE_DATE)) + ' - ' + TITLE + ' - ' + FORMAT(BUDGET, '0') 
	FROM MOVIE 
	WHERE COLLECTIONID = CO.COLLECTIONID 
	ORDER BY REVENUE DESC) AS [ENYUKSEKHASILAT],
	(SELECT TOP 1 CONVERT(VARCHAR, YEAR(RELEASE_DATE)) + ' - ' + TITLE + ' - ' + FORMAT(BUDGET, '0') 
	FROM MOVIE 
	WHERE COLLECTIONID = CO.COLLECTIONID 
	ORDER BY REVENUE) AS [ENDUSUKHASILAT]
FROM COLLECTION_ AS CO
ORDER BY 3 DESC

In [ ]:
# SORU 6
SELECT 
	PE.NAME_,
	JO.DEPARTMENT,
	JO.JOB,
	COUNT(*) AS [FILMSAYISI]
FROM PERSON AS PE
-- JOIN CAST_CREDIT AS CC1 ON CC1.PERSONID = PE.PERSONID
JOIN CREW_CREDIT AS CC2 ON CC2.PERSONID = PE.PERSONID
JOIN JOB AS JO ON JO.ID = CC2.JOBID
WHERE PE.PERSONID = 138
GROUP BY
	PE.NAME_,
	JO.DEPARTMENT,
	JO.JOB

In [ ]:
# SORU 7
SELECT 
	MC.COUNTRYCODE,
	CO.COUNTRY,
	COUNT(FILMID) AS [FILM SAYISI]
FROM MOVIE_COUNTRY AS MC
JOIN COUNTRY AS CO ON CO.CODE = MC.COUNTRYCODE
GROUP BY 
	COUNTRYCODE,
	CO.COUNTRY
ORDER BY 3 DESC

In [ ]:
# SORU 8
SELECT 
	MC.COUNTRYCODE AS [ULKE KODU],
	CO.COUNTRY AS [ULKE ADI],
	MO.TITLE,
	MO.ORIGINAL_TITLE
FROM MOVIE_COUNTRY AS MC
JOIN MOVIE AS MO ON MO.FILMID = MC.FILMID
JOIN COUNTRY AS CO ON CO.CODE = MC.COUNTRYCODE
WHERE 
	CO.CODE = 'JP'
	AND MO.FILMID IN (SELECT FILMID FROM MOVIE_COUNTRY WHERE COUNTRYCODE = 'US')

In [ ]:
# SORU 9
# OPTION 1
SELECT 
	GENRE,
	CONCAT([RANK], '-', TITLE),
	POPULARITY
FROM (
	SELECT 
		GENRE,
		TITLE,
		SUM(POPULARITY) AS POPULARITY,
		ROW_NUMBER() OVER (PARTITION BY GENRE ORDER BY SUM(POPULARITY) DESC) AS [RANK]
	FROM MOVIE AS MO
	JOIN MOVIE_GENRE AS MG ON MG.FILMID = MO.FILMID
	JOIN GENRE AS GE ON GE.GENREID = MG.GENREID
	GROUP BY
		GENRE,
		TITLE
) AS T
WHERE [RANK] < 6
ORDER BY
	1,
	3 DESC
    
# OPTION 2
SELECT 
	GE2.GENRE, 
	CONCAT([RANK], '-', T.TITLE),
	T.POPULARITY
FROM GENRE AS GE2
CROSS APPLY
(
	SELECT TOP 5
		GENRE,
		TITLE,
		SUM(POPULARITY) AS POPULARITY,
		ROW_NUMBER() OVER (ORDER BY SUM(POPULARITY) DESC) AS [RANK]
	FROM MOVIE AS MO
	JOIN MOVIE_GENRE AS MG ON MG.FILMID = MO.FILMID
	JOIN GENRE AS GE ON GE.GENREID = MG.GENREID
	WHERE GE.GENREID = GE2.GENREID
	GROUP BY
		GENRE,
		TITLE
) AS T